### Redis Client

In [1]:
import pandas as pd
import numpy as np

import sys
import os

if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))

from barchartacs import build_db
from IPython import display

import datetime
from dateutil.relativedelta import relativedelta
import io

from tqdm import tqdm,tqdm_notebook

from barchartacs import pg_pandas as pg
from barchartacs import schedule_it as sch

import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility

import ipdb,pdb
import traceback

import pandas_datareader.data as pdr
from scipy.stats import norm

import pyarrow as pa
import redis



/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
redis_port = 6379
redis_db = redis.Redis(host = 'localhost',port=6379,db=0)

In [3]:
def get_redis_df(key):
    context = pa.default_serialization_context()
    df = context.deserialize(redis_db.get(key))
    return df

### Get Data

In [6]:
#df_spy,df_vix,df_tnx,df_1yr_rate,df_div
df_spy = get_redis_df('df_spy')
df_vix = get_redis_df('df_vix')
df_tnx = get_redis_df('df_tnx')
df_1yr_rate = get_redis_df('df_1yr_rate')
df_div = get_redis_df('df_div')


In [7]:
df_spy.tail()

,high,low,open,close,volume,date,settle_date
7553,3079.760010,2965.659912,2993.760010,3066.590088,5740660000,2020-06-15,20200615
7554,3153.449951,3076.060059,3131.000000,3124.739990,5829240000,2020-06-16,20200616
7555,3141.159912,3108.030029,3136.129883,3113.489990,4549390000,2020-06-17,20200617
7556,3120.000000,3093.510010,3101.639893,3115.340088,4429030000,2020-06-18,20200618
7557,3155.530029,3083.110107,3140.290039,3097.739990,8327780000,2020-06-19,20200619


In [28]:
df_cuv_implied = get_redis_df('df_cuv_implied')

In [31]:
df_cuv_implied[df_cuv_implied.symbol.str.slice(0,2)=='CL']

,settle_date,iv,fprice,symbol
11528,20110103,0.507499,91.55,CLG11
11529,20110104,0.538744,89.38,CLG11
11530,20110105,0.512729,90.30,CLG11
11531,20110106,0.527733,88.38,CLG11
11532,20110107,0.519928,88.03,CLG11
...,...,...,...,...
23075,20200615,0.378903,38.43,CLZ20
23076,20200616,0.378089,39.51,CLZ20
23077,20200617,0.373733,38.90,CLZ20
23078,20200618,0.380135,39.63,CLZ20
